In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd
from stressor_utils import bin_layer
from stressor_utils import calculate_cell_area, resample_structured_grid

In [2]:
receptor_filename = r"H:\Projects\C1308_SEAT\SEAT_inputs\plugin-input\DEMO structured\receptor\grainsize_receptor.tif"
data = gdal.Open(receptor_filename)
img = data.GetRasterBand(1)
receptor_array = img.ReadAsArray()
receptor_array[receptor_array < 0] = 0
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = data.GetGeoTransform()
cols = data.RasterXSize
rows = data.RasterYSize
r_rows = np.arange(rows) * y_size + upper_left_y + (y_size / 2)
r_cols = np.arange(cols) * x_size + upper_left_x + (x_size / 2)
rx,ry = np.meshgrid(r_cols, r_rows)


In [3]:
stressor_filename = r"H:\Projects\C1308_SEAT\SEAT_inputs\plugin-output\Demo_WEC_structured\calculated_stressor.tif"
data = gdal.Open(stressor_filename)
img = data.GetRasterBand(1)
stressor_array = img.ReadAsArray()
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = data.GetGeoTransform()
cols = data.RasterXSize
rows = data.RasterYSize
r_rows = np.arange(rows) * y_size + upper_left_y + (y_size / 2)
r_cols = np.arange(cols) * x_size + upper_left_x + (x_size / 2)
rx,ry = np.meshgrid(r_cols, r_rows)

In [4]:
rrx, rry, sqarea = calculate_cell_area(rx, ry, latlon=True)

In [5]:
len(r_cols)

282

In [6]:
sqarea.shape

(191, 281)

In [7]:
rx2,ry2 = np.meshgrid(np.arange(0, len(r_cols), 1), np.arange(0, len(r_rows), 0.75))
rrx2, rry2, sqarea2 = calculate_cell_area(rx2, ry2, latlon=False)
sqarea2

array([[0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       ...,
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75],
       [0.75, 0.75, 0.75, ..., 0.75, 0.75, 0.75]])

In [8]:
DF = bin_layer(rx, ry, stressor_array, receptor=receptor_array, receptor_names=None, latlon=True)

h:\Projects\C1308_SEAT\Acoustics\seat-qgis-plugin\code\stressor_utils.py:252: RuntimeWarning: invalid value encountered in divide
  DATA[f'Area percent, receptor value {rval}'] = 100 * DATA[rcolname]/DATA[rcolname].sum()


In [9]:
DF

,bin start,bin end,bin center,"Area m2, receptor value 0.0","Area percent, receptor value 0.0","Area m2, receptor value 50.0","Area percent, receptor value 50.0","Area m2, receptor value 150.0","Area percent, receptor value 150.0","Area m2, receptor value 175.0",...,"Area m2, receptor value 2500.0","Area percent, receptor value 2500.0","Area m2, receptor value 2525.0","Area percent, receptor value 2525.0","Area m2, receptor value 2575.0","Area percent, receptor value 2575.0","Area m2, receptor value 2650.0","Area percent, receptor value 2650.0","Area m2, receptor value 5000.0","Area percent, receptor value 5000.0"
0,-1.322797,-1.208803,-1.265800,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,14144.427764,5.717952,0.000000,0.000000,0.000000,0.000000,7.072094e+03,0.091245,7.072094e+03,0.052575
1,-1.208803,-1.094809,-1.151806,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
2,-1.094809,-0.980815,-1.037812,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
3,-0.980815,-0.866820,-0.923817,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
4,-0.866820,-0.752826,-0.809823,0.000000e+00,0.000000,0.000000,0.000000,4.238423e+04,0.428744,0.000000,...,0.000000,0.000000,0.000000,0.000000,28256.315756,1.586984,0.000000e+00,0.000000,9.183351e+04,0.682699
5,-0.752826,-0.638832,-0.695829,0.000000e+00,0.000000,0.000000,0.000000,2.401952e+05,2.429732,0.000000,...,0.000000,0.000000,0.000000,0.000000,148354.754846,8.332179,1.130265e+05,1.458288,5.722307e+05,4.254018
6,-0.638832,-0.524837,-0.581835,0.000000e+00,0.000000,0.000000,0.000000,1.399016e+06,14.151968,0.000000,...,0.000000,0.000000,0.000000,0.000000,353257.805914,19.840331,4.521015e+05,5.833097,1.872202e+06,13.918126
7,-0.524837,-0.410843,-0.467840,7.762138e+04,0.394366,7057.971034,24.979022,2.056002e+06,20.797811,14115.942068,...,7056.413199,2.852589,0.000000,0.000000,713607.657525,40.078978,1.983988e+06,25.597782,4.696694e+06,34.915671
8,-0.410843,-0.296849,-0.353846,5.645047e+04,0.286804,21197.622869,75.020978,2.403289e+06,24.310843,169444.930938,...,14113.185937,5.705322,0.000000,0.000000,473408.845495,26.588480,2.244759e+06,28.962294,3.375223e+06,25.091733
9,-0.296849,-0.182854,-0.239852,4.233704e+04,0.215099,0.000000,0.000000,6.009139e+05,6.078638,0.000000,...,7072.213882,2.858976,7052.097251,20.002245,63618.233680,3.573047,1.749974e+06,22.578489,2.010688e+06,14.947649
